In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json

with open(r"FGCU vs FIU Women's Basketball Game Summary - November 13th, 2022 NCAA complete.htm", encoding="utf8") as file:
    soup = BeautifulSoup(file.read(), 'html.parser')

box = soup.find('div',id='gamecenterAppContent')

with open(r"FGCU vs FIU Women's Basketball Game Summary - November 13th, 2022 NCAA.com.html", encoding="utf8") as file:
    soup = BeautifulSoup(file.read(), 'html.parser')

pbp = soup.find('div',class_='gamecenter-tab-play-by-play')
pbp

FileNotFoundError: [Errno 2] No such file or directory: "FGCU vs FIU Women's Basketball Game Summary - November 13th, 2022 NCAA complete.htm"

In [17]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import requests

options = Options()
options.add_argument("--headless")

url = r'https://www.ncaa.com/game/6064407/'
driver = webdriver.Firefox(options=options)
driver.get(url)
print('a')
soup = BeautifulSoup(driver.page_source, 'html.parser')

#driver.quit()


a


In [11]:
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [5]:
driver.find_element(By.CLASS_NAME,'awayTeam-bg-primary_color').click()

In [6]:
driver.find_element(By.LINK_TEXT,'PLAY-BY-PLAY').click()

In [ ]:
pbp = soup.find('div',class_='gamecenter-tab-play-by-play')
pbp

In [8]:
team_names = [team for team in soup.find('div',class_='boxscore-team-selector').text.split('\n') if team]
team_map = {'away': team_names[0], 'home':team_names[1]}

In [9]:
def getCurrentLineup(input_row, team):
    """ Calculates the amount of time since a player last committed a foul

    Args:
        row (pd.Series): Play-by-play data row for a foul event

    Returns:
        float: Time in seconds since player's last foul
    """
    global df, starters, current_players
    row = input_row[1]
    if row.name == 0:
        return starters[team]
    else:
        description = row['description']
        prev_row = df.loc[row.name - 1]
        lineup = (current_players[team][-1]).copy()
        
        if 'Sub' in description and team_map[team] in description:
            
            subbing_description = row['description'].split()
            in_or_out = subbing_description[1]
            player = row['description'].split('-')[-1].upper()
            if in_or_out == 'out':
                lineup.remove(player)
                return lineup
            else:
                lineup.append(player)
                return lineup
        else:
            return lineup

In [13]:
categories = ['time', 'team', 'description', 'score', 'quarter']
data = {col:[] for col in categories}

quarters = pbp.find_all('div',class_='play-by-play-period-table')

for n, quarter in enumerate(quarters):
    table = quarter.contents[0].contents[3]

    for col in categories:
        if col == 'team':
            column_data = table.find_all('img')
            data[col].extend( [img.attrs['alt'].split()[0] for img in column_data] )
        elif col == 'quarter':
            data[col].extend( [n+1 for i in column_data] )
        else:
            column_data = table.find_all('td',class_=col)
            data[col].extend( [t.string.strip() for t in column_data] )
    

df = pd.DataFrame(data)

#df.to_json('fgcu.json')
df

,time,team,description,score,quarter
0,09:42,FGCU,Jumper MISSED by FGCU's Tishara Morehouse,,1
1,09:27,FIU,Turnover by FIU 's Kaliah Henderson,0-0,1
2,09:19,FGCU,Jumper by FGCU's Emma List,3-0,1
3,08:53,FIU,Turnover by FIU 's Ajae Yoakum,3-0,1
4,08:47,FIU,Personal Foul on FIU's Maria Torres (Tishara M...,3-0,1
...,...,...,...,...,...
399,01:05,FIU,Layup by FIU's Maria Torres,87-63,4
400,00:46,FGCU,Jumper by FGCU's Seneca Hackley,90-63,4
401,00:37,FIU,Jumper MISSED by FIU's Mihaela Lazic,,4
402,00:21,FGCU,Layup by FGCU's Maddie Antenucci,92-63,4


In [14]:

def get_starters(box):
    rows = box.contents[0].contents[1].find_all('tr')
    players = []

    for i, row in enumerate(rows):
        if i not in [0,len(rows)-1,len(rows)-2]:
            players.append(row.contents[1].string.strip())

    return players[0:5]

box_scores = {}
starters = {}

# Get the starting lineups for each team
for team in ['away', 'home']:
    driver.find_element(By.CLASS_NAME,f'{team}Team-bg-primary_color').click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    box_scores[team] = soup.find('div',id='gamecenterAppContent')

    starters[team] = get_starters(box_scores[team])

In [15]:
starters

{'away': ['10:00', '10:00', '10:00', '10:00', '10:00'],
 'home': ['10:00', '10:00', '10:00', '10:00', '10:00']}

In [ ]:
current_players

In [34]:
current_players = {'home':[], 'away':[]}

for team in ['home', 'away']:
    for row in df.iterrows():
        current_players[team].append(getCurrentLineup(row, team))

    df[f'{team}_current_lineup'] = current_players[team]
#df.to_json('fgcu.json')

In [33]:
df[~df['description'].str.contains('Sub') & [len(l)!=5 for l in current_players['away']]]

,time,team,description,score,quarter,home_current_lineup,away_current_lineup
